<a href="https://colab.research.google.com/github/vivirocha/Bootcamp_DataScience/blob/main/Desafio_Desenvolvimento_de_Solu%C3%A7%C3%B5es_Utilizando_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Bootcamp Cientista de Dados - IGTI**

Desafio - Módulo 2

*Desenvolvendo Soluções Utilizando Apache Spark*

**Enunciado** <br>
Doenças ligadas ao coração afetam milhões de pessoas ao redor do mundo e, segundo a Organização Mundial da Saúde (OMS), são a segunda principal causa de morte da população mundial. Como cientista de dados, você foi contratado para criar um modelo preditivo que, a partir de dados de pacientes como idade, gênero, nível de glicose, se o paciente fuma ou não, vai prever se aquele paciente terá um derrame cerebral ou não. <br>
Você tem acesso a um arquivo que possui atributos de pacientes e um atributo “stroke” (derrame), que indica se aquele paciente sofreu um evento de derrame ou não. <br>
O conjunto de dados está disponível em: https://dcc.ufmg.br/~pcalais/stroke_data.csv

<br> 

Os dados estão distribuídos nas seguintes colunas: <br>
**id** = valores únicos. <br>
**gender** = feminino, masculino ou outro. <br>
**age** = idade <br>
**hypertensio.** = valores binários que mostram se o paciente é ou não hipertenso(0 = não, 1 = sim). <br>
**heart_disease** =  valores binários que mostram se o paciente tem ou não alguma doença cardiáca (0 = não, 1 = sim). <br>
**ever_married** = Sim ou Não. <br>
**work_type** = "children", "Govt_jov", "Never_worked", "Private" ou "Self-employed". <br>
**Residence_type** = Rural ou urbana. <br>
**avg_glucose_level** = nível médio de glucose no sangue. <br>
**bmi** = índice de massa corporal. <br>
**smoking_status** = "formerly smoked", "never smoked", "smokes" ou "Unknown". <br> 
**stroke** = valores binários que mostram se o paciente teve derrame (1) ou não teve (0).

In [34]:
#Instalando spark
!pip install spark

In [35]:
#Instalando pyspark
!pip install pyspark

In [36]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName('Desafio_SolucoesSpark') \
                    .getOrCreate()

In [37]:
#Lendo o arquivo .csv e criando um dataframe a partir dele.
stroke = spark.read.csv('stroke_data.csv', 
                        sep=',', 
                        inferSchema=True,
                        header=True).cache()

In [38]:
stroke.show()

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|      Private|         Rural|           154.24| 33.7|   never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  5|Female|82.0|           0|            0|         Yes|     

Quantos registros existem no arquivo?

In [39]:
stroke.count()

67135

Quantas colunas existem no arquivo? Quantas são numéricas? Ao ler o arquivo com spark.read.csv, habilite inferSchema=True. Use a função printSchema() da API de Dataframes.

In [40]:
stroke.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [41]:
#Importando col a partir do pyspark.sql.functions
from pyspark.sql.functions import col

In [42]:
stroke = stroke.select(col("0").alias("id"), col("gender").alias("gender"), col("age").alias("age"), col("hypertension").alias("hypertension"),col("heart_disease").alias("heart_disease"),col("ever_married").alias("ever_married"),col("work_type").alias("work_type"),col("Residence_type").alias("Residence_type"),col("avg_glucose_level").alias("avg_glucose_level"),col("bmi").alias("bmi"), col("smoking_status").alias("smoking_status"),col("stroke").alias("stroke"))
stroke.show()

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
| id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|      Private|         Rural|           154.24| 33.7|   never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  5|Female|82.0|           0|            0|         Yes|     

No conjunto de dados, quantos pacientes sofreram e não sofreram derrame (stroke), respectivamente?
Para descobrirmos a resposta para esta pergunta, primeiro vamos alterar o tipo das colunas para que as que tenham valores binários, tenham o tipo booleando.

In [43]:
#Alterando o tipo das colunas
from pyspark.sql.types import StringType, DateType, FloatType
from pyspark.sql.types import BooleanType, IntegerType

In [44]:
stroke = stroke\
.withColumn('Id',
            stroke['Id'].cast(IntegerType()))\
.withColumn("gender",
            stroke["gender"]
            .cast(StringType()))\
.withColumn("age",stroke["age"].cast(IntegerType()))    \
.withColumn("hypertension", stroke["hypertension"].cast(BooleanType()))    \
.withColumn("heart_disease",stroke["heart_disease"].cast(BooleanType()))    \
.withColumn("ever_married",stroke["ever_married"].cast(StringType()))    \
.withColumn("work_type",stroke["work_type"].cast(StringType()))    \
.withColumn("Residence_type",stroke["Residence_type"].cast(StringType()))    \
.withColumn("avg_glucose_level",stroke["avg_glucose_level"].cast(FloatType()))    \
.withColumn("bmi",stroke["bmi"].cast(FloatType()))    \
.withColumn("smoking_status",stroke["smoking_status"].cast(StringType()))    \
.withColumn("stroke",stroke["stroke"].cast(BooleanType()))    \

In [45]:
stroke.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- hypertension: boolean (nullable = true)
 |-- heart_disease: boolean (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: float (nullable = true)
 |-- bmi: float (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: boolean (nullable = true)



In [46]:
stroke.select('stroke').where(stroke.stroke=="1").count()

40287

In [47]:
stroke.select('stroke').where(stroke.stroke=="0").count()

26848

A partir do dataframe, crie uma tabela temporária usando df.createOrReplaceTempView('table') e, a seguir, use spark.sql para escrever uma consulta SQL que obtenha quantos pacientes tiveram derrame por tipo de trabalho (work_type). Quantos pacientes que sofreram derrame trabalhavam, respectivamente, no setor privado, de forma independente, no governo e quantas são crianças?

In [48]:
stroke.createOrReplaceTempView('table') 

In [49]:
from pyspark.sql.functions import *

In [67]:
private = stroke.where(stroke.work_type == 'Private')
private.count()

37806

In [65]:
Self_employed = stroke.where(stroke.work_type == 'Self-employed')
Self_employed.count()

14736

In [69]:
Govt_job = stroke.where(stroke.work_type == 'Govt_job')
Govt_job.count()

8260

In [71]:
children = stroke.where(stroke.work_type == 'children')
children.count()

6156